This notebook describes the scraping of MEI data from the NOAA page.

In [1]:
import pandas as pd
import re
from IPython.core.display import HTML, display
from bs4 import BeautifulSoup as bs
import requests
display(HTML("<style>.container {width:90% !important}</style>"))

In [6]:
url = 'https://www.esrl.noaa.gov/psd/enso/mei/table.html'
response = requests.get(url)
textsoup = bs(response.text, 'html5lib')

In [2]:
table_re = re.compile(r'(^YEAR.+|^19.+|^20.+)')

In [5]:
tbl_str.splitlines()

['MEI Index (current update: 6 March 2018)',
 '',
 'Bimonthly MEI values (in 1/1000 of standard deviations), starting with Dec1949/Jan1950, thru last',
 'month.  More information on the MEI can be found on the MEI homepage.',
 'Missing values are left blank.  Note that values can still change with each monthly update, even ',
 'though such changes are typically smaller than +/-0.1.  All values are normalized for each bimonthly',
 'season so that the 44 values from 1950 to 1993 have an average of zero and a standard deviation of "1".  ',
 "Responses to 'FAQs' can be found below this table:",
 '',
 'YEAR    DECJAN  JANFEB  FEBMAR  MARAPR  APRMAY  MAYJUN  JUNJUL  JULAUG  AUGSEP  SEPOCT  OCTNOV  NOVDEC',
 '1950\t-1.03\t-1.133\t-1.312\t-1.098\t-1.445\t-1.376\t-1.267\t-1.03\t-.597\t-.406\t-1.138\t-1.235',
 '1951\t-1.049\t-1.152\t-1.204\t-.544\t-.374\t.319\t.676\t.842\t.773\t.768\t.726\t.504',
 '1952\t.433\t.138\t.047\t.198\t-.307\t-.722\t-.307\t-.358\t.347\t.306\t-.328\t-.098']

In [3]:
# This is an improperly formatted html page lacking in proper tags for the table.
# Below, I resort to "manual" scraping.
headerFound = False
cntr = 0
for sti in textsoup.stripped_strings:
    for s in sti.splitlines():
        if table_re.match(s):
            mylist = re.sub('\t|\s+', ',', table_re.findall(s)[0]).split(',')
            if headerFound:
                df = df.append({col: elem for col, elem in zip(df.columns, mylist)}, ignore_index=True)
            else:
                df = pd.DataFrame(columns=mylist)
                headerFound=True

In [86]:
df.iloc[:, 1:] = df.iloc[:, 1:].astype('float')

In [4]:
df.head()

,YEAR,DECJAN,JANFEB,FEBMAR,MARAPR,APRMAY,MAYJUN,JUNJUL,JULAUG,AUGSEP,SEPOCT,OCTNOV,NOVDEC
0,1950,-1.03,-1.163,-1.312,-1.098,-1.445,-1.376,-1.267,-1.03,-.597,-.406,-1.138,-1.235
1,1951,-1.049,-1.183,-1.204,-.544,-.374,.319,.676,.842,.773,.768,.726,.504
2,1952,.433,.117,.047,.198,-.307,-.722,-.307,-.358,.347,.306,-.328,-.098
3,1953,.044,.377,.257,.668,.773,.226,.379,.228,.527,.124,.099,.351
4,1954,-.036,-.048,.147,-.634,-1.478,-1.528,-1.356,-1.446,-1.138,-1.32,-1.113,-1.088


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Data columns (total 13 columns):
YEAR      69 non-null object
DECJAN    69 non-null object
JANFEB    68 non-null object
FEBMAR    68 non-null object
MARAPR    68 non-null object
APRMAY    68 non-null object
MAYJUN    68 non-null object
JUNJUL    68 non-null object
JULAUG    68 non-null object
AUGSEP    68 non-null object
SEPOCT    68 non-null object
OCTNOV    68 non-null object
NOVDEC    68 non-null object
dtypes: object(13)
memory usage: 7.1+ KB


The next bit is to unpack the bimonthly data format and get a dataframe containing monthly data (columns) for each year (rows)

In [11]:
# set up a shifted (by one year) column for the DECJAN data
df['DECJAN_SH'] = df.DECJAN.shift(-1)

In [12]:
df.head()

,YEAR,DECJAN,JANFEB,FEBMAR,MARAPR,APRMAY,MAYJUN,JUNJUL,JULAUG,AUGSEP,SEPOCT,OCTNOV,NOVDEC,DECJAN_SH
0,1950,-1.03,-1.163,-1.312,-1.098,-1.445,-1.376,-1.267,-1.03,-.597,-.406,-1.138,-1.235,-1.049
1,1951,-1.049,-1.183,-1.204,-.544,-.374,.319,.676,.842,.773,.768,.726,.504,.433
2,1952,.433,.117,.047,.198,-.307,-.722,-.307,-.358,.347,.306,-.328,-.098,.044
3,1953,.044,.377,.257,.668,.773,.226,.379,.228,.527,.124,.099,.351,-.036
4,1954,-.036,-.048,.147,-.634,-1.478,-1.528,-1.356,-1.446,-1.138,-1.32,-1.113,-1.088,-.74


In [13]:
# average data for overlapping months rolling mean over overlapping months. Skip the first DECJAN column
df2 = df.rolling(window=2, axis=1).mean().iloc[:, 2:]

In [14]:
# change the column names to single month - use abbrev. 3-letter names.
df2.rename(columns={k: k[:3] for k in df.columns}, inplace=True)

In [15]:
# reintroduce the YEAR column
df2.insert(0, 'YEAR', df.YEAR)

In [16]:
df2.head()

,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
0,1950,-1.0965,-1.2375,-1.2050,-1.2715,-1.4105,-1.3215,-1.1485,-0.8135,-0.5015,-0.7720,-1.1865,-1.1420
1,1951,-1.1160,-1.1935,-0.8740,-0.4590,-0.0275,0.4975,0.7590,0.8075,0.7705,0.7470,0.6150,0.4685
2,1952,0.2750,0.0820,0.1225,-0.0545,-0.5145,-0.5145,-0.3325,-0.0055,0.3265,-0.0110,-0.2130,-0.0270
3,1953,0.2105,0.3170,0.4625,0.7205,0.4995,0.3025,0.3035,0.3775,0.3255,0.1115,0.2250,0.1575
4,1954,-0.0420,0.0495,-0.2435,-1.0560,-1.5030,-1.4420,-1.4010,-1.2920,-1.2290,-1.2165,-1.1005,-0.9140


In [17]:
# format column names to match satellite dataframes used in subsequent notebooks
df2.rename(columns={k: '%s%s' %(k[0],k[1:].lower()) for k in df2.columns},inplace=True)

In [18]:
df2.head()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1950,-1.0965,-1.2375,-1.2050,-1.2715,-1.4105,-1.3215,-1.1485,-0.8135,-0.5015,-0.7720,-1.1865,-1.1420
1,1951,-1.1160,-1.1935,-0.8740,-0.4590,-0.0275,0.4975,0.7590,0.8075,0.7705,0.7470,0.6150,0.4685
2,1952,0.2750,0.0820,0.1225,-0.0545,-0.5145,-0.5145,-0.3325,-0.0055,0.3265,-0.0110,-0.2130,-0.0270
3,1953,0.2105,0.3170,0.4625,0.7205,0.4995,0.3025,0.3035,0.3775,0.3255,0.1115,0.2250,0.1575
4,1954,-0.0420,0.0495,-0.2435,-1.0560,-1.5030,-1.4420,-1.4010,-1.2920,-1.2290,-1.2165,-1.1005,-0.9140


In [19]:
# create a new dataframe such that the data is index by yearmonth
# first, create a new column containing months
df3 = pd.melt(df2, id_vars=["Year"], var_name="Month", value_name="MEI",)

In [20]:
df3.head()

,Year,Month,MEI
0,1950,Jan,-1.0965
1,1951,Jan,-1.1160
2,1952,Jan,0.2750
3,1953,Jan,0.2105
4,1954,Jan,-0.0420


In [21]:
# create a new column where year and month are aggregated
df3['date'] = df3[['Year', 'Month']].apply(lambda x: ' '.join(x), axis=1)

In [22]:
df3.head()

,Year,Month,MEI,date
0,1950,Jan,-1.0965,1950 Jan
1,1951,Jan,-1.1160,1951 Jan
2,1952,Jan,0.2750,1952 Jan
3,1953,Jan,0.2105,1953 Jan
4,1954,Jan,-0.0420,1954 Jan


In [23]:
# convert date column data into datetime objects
df3['datetime'] = pd.to_datetime(df3.date, format='%Y %b')

In [24]:
# set datetime column as index
df3 = df3.set_index('datetime').sort_index()

In [25]:
# drop unneeded columns
# collect data from 1997 on to match satellite data
df3.drop(['Year', 'Month', 'date'], axis=1, inplace=True)
df3 = df3['1997':]
df3.head()

,MEI
datetime,
1997-01-01,-0.5505
1997-02-01,-0.4365
1997-03-01,0.1455
1997-04-01,0.8415
1997-05-01,1.7135


In [147]:
# pickle this final dataframe
df3.to_pickle('./PklJar/dfMEI.pkl')